<a href="https://colab.research.google.com/github/lauragj95/landmarks-navigation-and-recognition/blob/main/SRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/My Drive/src/Gan/SRGAN_Modules')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Copio el contenido de esta carpeta en la VM para que vaya mas rapido (Tarda mucho en copiar)
!cp -r "/content/drive/My Drive/src/Gan/People_faces_256x256/" ./People_faces_256x256

In [ ]:
from Network import Generator, Discriminator
import Utils, Utils_model
from Utils_model import VGG_LOSS
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tqdm import tqdm
import numpy as np
import argparse
import pickle
import tensorflow as tf
import time

In [ ]:
tf.__version__

'2.2.0-rc3'

In [ ]:
np.random.seed(10)
# LR images shape target
target = 32
# Remember to change image shape if you are having different size of images
image_shape = (256,256,3)
# Better to use downscale factor as 4
downscale_factor = int(image_shape[0]/target)

LAMBDA = 100
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
downscale_factor

8

In [ ]:
# Combined network
def get_gan_network(discriminator, shape, generator, optimizer, loss):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    

    gan_output = discriminator(x)

    gan = Model(inputs=gan_input, outputs=[x, gan_output])
    gan.compile(loss=[loss, "binary_crossentropy"],
                loss_weights=[1., 1e-3],
                optimizer=optimizer)
    ###Añadido
    gan.summary()
    discriminator.trainable = True
    ####------
    return gan

In [ ]:
def save(gan, generator, discriminator, optimizer, loss, model_path, epoch):

    discriminator.trainable = False
    gan.save(model_path + '%d_gan_model' % epoch, save_format = 'tf')
    gan.compile(loss=[loss, "binary_crossentropy"], optimizer=optimizer)
    discriminator.trainable = True

    discriminator.save(model_path + '%d_dis_model' % epoch, save_format = 'tf')
    discriminator.compile(loss="binary_crossentropy", optimizer=optimizer)

    generator.save(model_path + '%d_gen_model' % epoch, save_format = 'tf')
    generator.compile(loss=loss, optimizer=optimizer)


In [ ]:
def custom_vgg_loss(y_true, y_pred):

    vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=image_shape)
    vgg19.trainable = False
    # Make trainable as False
    for l in vgg19.layers:
        l.trainable = False
    #tf.config.experimental_run_functions_eagerly(True)
    model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
    model.trainable = False

    return K.mean(K.square(model(y_true) - model(y_pred)))

In [ ]:
def load(model_path, epoch):

    loss = VGG_LOSS(image_shape)  

    generator = load_model(model_path + '%d_gen_model' % epoch, custom_objects={ 'vgg_loss': loss.vgg_loss })
    discriminator = load_model(model_path + '%d_dis_model' % epoch)
    gan = load_model(model_path + '%d_gan_model' % epoch, custom_objects={ 'vgg_loss': loss.vgg_loss })

    generator.summary()
    discriminator.summary()
    gan.summary()
    return gan, generator, discriminator

In [ ]:
#gan, generator, discriminator = load("/content/drive/My Drive/src/Gan/SRGAN_model/", 25)

**Discriminator loss**
  * The discriminator loss function takes 2 inputs; **real images, generated images**
  * real_loss is a sigmoid cross entropy loss of the **real images** and an **array of ones(since these are the real images)**
  * generated_loss is a sigmoid cross entropy loss of the **generated images** and an **array of zeros(since these are the fake images)**
  * Then the total_loss is the sum of real_loss and the generated_loss



In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss
  return total_disc_loss


 **Generator loss**
  * It is a sigmoid cross entropy loss of the generated images and an **array of ones**.
  * The [paper](https://arxiv.org/abs/1611.07004) also includes L1 loss which is MAE (mean absolute error) between the generated image and the target image.
  * This allows the generated image to become structurally similar to the target image.
  * The formula to calculate the total generator loss = gan_loss + LAMBDA * l1_loss, where LAMBDA = 100. This value was decided by the authors of the [paper](https://arxiv.org/abs/1611.07004).


In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss


In [ ]:
#No usada, no funciona dado que keras no guarda el estado de las capas del discriminador
def save_optimizer_states(model, name, model_save_dir):

  symbolic_weights = getattr(model.optimizer, 'weights')
  weight_values = K.batch_get_value(symbolic_weights)
  with open(model_save_dir + name, 'wb') as f:
      pickle.dump(weight_values, f)


**Training**
* No need to iterate over the dataset. We do all the calculations at the same time.
* The generator gets the input image and we get a generated output.
* The discriminator receives the input_image and the generated image as the first input. The second input is the input_image and the target_image.
* Next, we calculate the generator and the discriminator loss.
* Then, we calculate the gradients of loss with respect to both the generator and the discriminator variables(inputs) and apply those to the optimizer.


In [ ]:
@tf.function
def train_step(shape_lr, input_image, target, g_loss_metric, d_loss_metric, generator, discriminator, generator_optimizer, discriminator_optimizer):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    print(input_image)
    gen_output = generator(input_image, training=True)

    disc_real_output = discriminator(target, training=True)
    disc_generated_output = discriminator(target, training=True)

    gen_loss = generator_loss(disc_generated_output, gen_output, target)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  # Update the metrics
  g_loss_metric.update_state(gen_loss)
  d_loss_metric.update_state(disc_loss)
  # accuracy_metric.update_state(tf.ones_like(gen_output), gen_output)
  generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))


# **`Entrenamiento (funcion antigua)`**

In [ ]:
def old_train(x_train_lr, x_train_hr, x_test_lr, x_test_hr, epochs, batch_size, output_dir, model_save_dir, weights_save_dir):
  loss = VGG_LOSS(image_shape)  
    
  batch_count = int(x_train_hr.shape[0] / batch_size)
  #### SI LAS IMAGENES NO SON CUADRADAS ESTO DEBERIA CAMBIAR
  shape_lr = (image_shape[0]//downscale_factor, image_shape[1]//downscale_factor, image_shape[2])
  shape_hr = x_train_hr[0].shape
  ####

  gen = Generator(shape_lr, shape_hr).generator()
  dis = Discriminator(image_shape).discriminator()

  optimizer = Utils_model.get_optimizer()
  gen.compile(loss=loss.vgg_loss, optimizer=optimizer)
  dis.compile(loss="binary_crossentropy", optimizer=optimizer)

  gan = get_gan_network(dis, shape_lr, gen, optimizer, loss.vgg_loss)
  
  loss_file = open(model_save_dir + '/losses.txt' , 'w+')
  loss_file.close()

  for e in range(1, epochs+1):
      print ('-'*15, 'Epoch %d' % e, '-'*15)
      for _ in tqdm(range(batch_count)):
          
          rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
          
          image_batch_hr = x_train_hr[rand_nums]
          image_batch_lr = x_train_lr[rand_nums]

          generated_images_sr = gen.predict(image_batch_lr)

          real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
          fake_data_Y = np.random.random_sample(batch_size)*0.2
          
          dis.trainable = True
          
          d_loss_real = dis.train_on_batch(image_batch_hr, real_data_Y)
          d_loss_fake = dis.train_on_batch(generated_images_sr, fake_data_Y)
          discriminator_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
          
          rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
          image_batch_hr = x_train_hr[rand_nums]
          image_batch_lr = x_train_lr[rand_nums]

          gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
          dis.trainable = False
          gan_loss = gan.train_on_batch(image_batch_lr, [image_batch_hr, gan_Y])
          
          
      print("discriminator_loss : %f" % discriminator_loss)
      print("gan_loss :", gan_loss)
      gan_loss = str(gan_loss)
      
      loss_file = open(model_save_dir + 'losses.txt' , 'a')
      loss_file.write('epoch%d : gan_loss = %s ; discriminator_loss = %f\n' %(e, gan_loss, discriminator_loss) )
      loss_file.close()

      if e == 1 or e % 5 == 0:
          Utils.plot_generated_images(output_dir, e, gen, x_test_hr, x_test_lr)

          save(gan, gen, dis, optimizer, loss.vgg_loss, model_save_dir, e)

      if e % 500 == 0 or e == epochs+1:
          Utils.plot_generated_images(output_dir, e, gen, x_test_hr, x_test_lr)

          save(gan, gen, dis, optimizer, loss.vgg_loss, model_save_dir, e)
        
          

#  ENTRENAMIENTO 

In [ ]:
def train(one_sample, x_train_lr, x_train_hr, x_test_lr, x_test_hr, epochs, batch_size, output_dir, model_save_dir, weights_save_dir):

    batch_count = int(x_train_hr.shape[0] / batch_size)
    image_batch_hr = tf.split(x_train_hr, num_or_size_splits = batch_count)    
    image_batch_lr = tf.split(x_test_lr, num_or_size_splits = batch_count)


    shape_lr = (image_shape[0]//downscale_factor, image_shape[1]//downscale_factor, image_shape[2])
    shape_hr = x_train_hr[0].shape
    print(type(one_sample))
    # Create the metrics
    g_loss_log = []
    d_loss_log = []
    g_loss_metric = tf.keras.metrics.Mean(name='g_train_loss')
    d_loss_metric = tf.keras.metrics.Mean(name='d_train_loss')
    accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    generator = Generator()
    generator_out = generator(one_sample, training = False)

    discriminator = Discriminator()
    discriminator_out = discriminator(shape_hr, training = False)

    checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                              discriminator_optimizer=discriminator_optimizer,
                              generator=generator, discriminator=discriminator)
    
    for epoch in range(epochs):
      start = time.time()
      # Reset the metrics
      g_loss_metric.reset_states()
      d_loss_metric.reset_states()
      counter = 0

      for i in tqdm(range(batch_count)):
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            
            image_batch_hr_slice = image_batch_hr[i]
            image_batch_lr_slice = image_batch_lr[i]
            
            for image_hr, image_lr in zip(image_batch_hr_slice, image_batch_lr_slice):
              train_step(shape_lr, image_lr, image_hr, g_loss_metric, d_loss_metric, 
                        generator, discriminator, generator_optimizer, discriminator_optimizer)

      # Get the metric results
      g_mean_loss = g_loss_metric.result()
      d_mean_loss = d_loss_metric.result()
      g_loss_log.append([g_mean_loss])
      d_loss_log.append([d_mean_loss])
      # mean_accuracy = accuracy_metric.result()

      print('Epoch: ', epoch)
      print('  loss (g) (d) (g+d):     {:.3f}, {:.3f}, {:.3f}'.format(g_mean_loss, d_mean_loss, g_mean_loss + d_mean_loss))

      if (epoch) % 5 == 0 or epoch == 1:
          Utils.plot_generated_images(output_dir, e, gen, x_test_hr, x_test_lr)
          plot_losses(g_loss_log, d_loss_log, epoch=epoch, dataset='dataset')

          checkpoint.save(file_prefix = os.path.join(output_dir, 'training_checkpoints', "ckpt"))

      print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                          time.time()-start))         

# **`Continuar entrenamiento`**

In [ ]:

##NO TENER EN CUENTA. SE TIENE QUE MODIFICAR
def resume_training(current_epoch, x_train_lr, x_train_hr, x_test_lr, x_test_hr, epochs, batch_size, output_dir, model_save_dir):

    batch_count = int(x_train_hr.shape[0] / batch_size)
    #### SI LAS IMAGENES NO SON CUADRADAS ESTO DEBERIA CAMBIAR
    shape_lr = (image_shape[0]//downscale_factor, image_shape[1]//downscale_factor, image_shape[2])
    shape_hr = x_train_hr[0].shape

    #gan, generator, discriminator = load(model_save_dir, current_epoch)
      


    loss = VGG_LOSS(image_shape)  


    generator = load_model(model_save_dir + '%d_gen_model' % current_epoch, custom_objects={ 'vgg_loss': loss.vgg_loss })

    discriminator = load_model(model_save_dir + '%d_dis_model' % current_epoch)

    gan = load_model(model_save_dir + '%d_gan_model' % current_epoch, custom_objects={ 'vgg_loss': loss.vgg_loss })



    #generator = load_model(model_path + '%d_gen_model' % epoch, custom_objects={ 'vgg_loss': loss.vgg_loss })
    #discriminator = load_model(model_path + '%d_dis_model' % epoch)
    #gan = load_model(model_path + '%d_gan_model' % epoch, custom_objects={ 'vgg_loss': loss.vgg_loss })

    generator.summary()
    discriminator.summary()
    gan.summary()

    loss_file = open(model_save_dir + 'losses.txt' , 'w+')
    loss_file.close()

    for e in range(current_epoch+1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batch_count)):
            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]

            generated_images_sr = generator.predict(image_batch_lr)

            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake_data_Y = np.random.random_sample(batch_size)*0.2
            
            discriminator.trainable = True
            
            d_loss_real = discriminator.train_on_batch(image_batch_hr, real_data_Y)
            d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
            discriminator_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]

            gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            discriminator.trainable = False
            gan_loss = gan.train_on_batch(image_batch_lr, [image_batch_hr, gan_Y])
            
            
        print("discriminator_loss : %f" % discriminator_loss)
        print("gan_loss :", gan_loss)
        gan_loss = str(gan_loss)
        
        loss_file = open(model_save_dir + 'losses.txt' , 'a')
        loss_file.write('epoch%d : gan_loss = %s ; discriminator_loss = %f\n' %(e, gan_loss, discriminator_loss) )
        loss_file.close()

        if e == 1 or e % 5 == 0:
            Utils.plot_generated_images(output_dir, e, generator, x_test_hr, x_test_lr)

            save(gan, generator, discriminator, model_save_dir, e)


        if e % 500 == 0 or e == epochs+1:
            Utils.plot_generated_images(output_dir, e, generator, x_test_hr, x_test_lr)

            save(gan, generator, discriminator, model_save_dir, e)


# **`Cargar datos almacenados en la VM`**

In [ ]:
x_train_lr, x_train_hr, x_test_lr, x_test_hr = Utils.old_load_training_data('./People_faces_256x256/', '.jpg', 0.8, downscale_factor) 
print('#'*10 + ' LOADED ' + '#'*10) 

  2%|#                                                                        |

Getting images paths...
Done
Reading images...


100%|#########################################################################|


Done
Images loaded!
Total images: 5000
Number of train images:4000
Downscale factor: 8


# **`Cargar datos almacenados en la nube`**

In [ ]:
    #x_train_lr, x_train_hr, x_test_lr, x_test_hr = Utils.load_training_data('/content/drive/My Drive/src/Gan/People_faces_256x256/', '.jpg', 0.8, downscale_factor) 
    #print('#'*10 + ' LOADED ' + '#'*10)

# **`Entrenar`**

In [ ]:
train(one_sample, x_train_lr, x_train_hr, x_test_lr, x_test_hr, 100, 20, "/content/drive/My Drive/src/Gan/SRGAN_output/", "/content/drive/My Drive/src/Gan/SRGAN_model/", "/content/drive/My Drive/src/Gan/SRGAN_weights/")

# **`Continuar entrenamiento`**

In [ ]:
#resume_training(20, x_train_lr, x_train_hr, x_test_lr, x_test_hr, 100, 10, "/content/drive/My Drive/src/Gan/SRGAN_output/", "/content/drive/My Drive/src/Gan/SRGAN_model/")